In [11]:
# import libraries
import pandas as pd
import numpy as np
import glob, os
data='../data/'

In [15]:
# download data
hip=pd.read_csv(data+'hipsci_files.tsv',sep='\t',index_col=0) # hipsci_files.tsv file is exported from https://www.hipsci.org/lines/#/lines 
ftps=hip['FTP url'].values
n=1                           # set n equal to 330 to download all the data 
for ftp in ftps[0:n]: 
    !wget -nc -P {data}/tpm {ftp+'*.tsv'}

--2021-04-15 19:38:54--  ftp://ftp.sra.ebi.ac.uk/vol1/ERZ267/ERZ267062/*.tsv
           => ‘../data//tpm/.listing’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/ERZ267/ERZ267062 ... done.
==> PASV ... done.    ==> LIST ... done.

    [ <=>                                   ] 330         --.-K/s   in 0.01s   

2021-04-15 19:38:55 (31.0 KB/s) - ‘../data//tpm/.listing’ saved [330]

Removed ‘../data//tpm/.listing’.
File ‘../data//tpm/HPSI0114i-bezi_1.GRCh37.75.cdna.kallisto.transcripts.abundance.rnaseq.20150415.tsv’ already there; not retrieving.


In [16]:
# compine them into a list of list
ips=[]
cells=[]
for file in glob.glob(data+"tpm/*.tsv"):
    cells.append(file.split('.GRCh37.')[0].split('tpm/')[1])
    ip=pd.read_csv(file,sep='\t',index_col=0)[['tpm']].round(2)  
    ip=ip.tpm.values
    ips.append(ip)
ind=pd.read_csv(file,sep='\t',index_col=0).index

In [17]:
# convert it into pandas
df=pd.DataFrame(ips)
df.columns=ind
df.index=cells
df=df.T
df.shape

(179659, 330)

In [18]:
#get ENS to Gene mapping data and mapp
ens=pd.read_csv(data+'mart_export.txt.gz',sep='\t',index_col=1) # mart_export file exported from http://www.ensembl.org/biomart/martview/
ens=ens.rename(columns={'Gene name':'gene','Gene stable ID':'id'})
print(ens.shape)
df=df.merge(ens[['gene']],left_index=True,right_index=True)

(215170, 2)


In [19]:
#aggregate transcripts to get gene level data and write it into a combined table
dg=df.groupby('gene').sum()
dg.T.to_csv(data+'ips_tpm.gz', compression='gzip')

In [20]:
#top few rows
dg.head()

,HPSI1213i-xuja_2,HPSI0614i-voce_2,HPSI0814i-doao_2,HPSI0314i-sojd_3,HPSI0114i-lexy_2,HPSI1013i-jufd_2,HPSI0215i-deyz_3,HPSI0514i-sohd_2,HPSI0714i-kute_4,HPSI0115i-payf_1,...,HPSI0214i-kucg_2,HPSI0114i-iisa_3,HPSI0114i-vabj_3,HPSI0115i-sehp_3,HPSI0314i-qonc_2,HPSI0214i-datg_2,HPSI0114i-kolf_3,HPSI0115i-vazt_2,HPSI0114i-iisa_1,HPSI0714i-nufh_3
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,21.10,7.71,19.51,17.89,14.99,15.65,19.42,13.19,15.50,15.96,...,20.23,18.90,25.67,31.10,23.66,22.48,22.97,13.43,21.35,13.07
A1CF,0.05,0.03,0.11,0.02,0.01,0.04,0.09,0.07,0.01,0.04,...,0.04,0.17,0.06,0.04,0.11,0.04,0.06,0.10,0.07,0.09
A2M,2.35,1.47,1.27,0.90,0.33,0.51,0.88,1.70,2.31,0.95,...,2.68,0.32,0.97,0.70,1.37,1.46,1.29,1.66,0.24,0.23
A2ML1,1.59,4.97,2.48,3.83,1.30,3.47,3.07,2.95,3.70,1.70,...,3.22,2.17,3.55,1.43,3.38,2.80,2.08,2.97,2.03,5.43
A2MP1,0.57,0.17,0.41,0.16,0.05,0.18,0.15,0.26,0.22,0.24,...,0.30,0.18,0.12,0.17,0.27,0.16,0.23,0.18,0.20,0.08
